# Instagram Agent Demo

This notebook demonstrates how to use the `ig_agent` library to generate Instagram carousel posts with AI.

## 1. Setup and Installation

First, ensure you have all the necessary dependencies installed:

In [1]:
# Install dependencies if needed
!pip install -q langchain_xai cairosvg
!pip install -q -e .

## 2. Basic Usage - Direct API

The simplest way to use the Instagram Agent is through its direct API, which provides methods for each step of the content generation and posting process.

In [2]:
import os
import logging
from pathlib import Path
from IPython.display import Image, display, HTML
from langchain_xai import ChatXAI
from ig_agent import InstagramAgent

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("ig_agent_demo")

In [3]:
# Create base output directory 
# Now outputs will be stored in datetime subdirectories
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Initialize language model
# You can use any LangChain compatible LLM here
llm = ChatXAI(model="grok-3-mini-beta")

# Initialize the Instagram agent
# Output will automatically be stored in a datetime-based directory structure
# For example: output/2025-04-20/20250420_154900/
agent = InstagramAgent(
    llm=llm,
    output_dir=output_dir,
    # Optional: provide credentials for notification and posting
    email_user=os.environ.get("EMAIL_USER"),
    email_password=os.environ.get("EMAIL_PASSWORD"),
    instagram_username=os.environ.get("INSTAGRAM_USERNAME"),
    instagram_password=os.environ.get("INSTAGRAM_PASSWORD")
)

### 2.1 Generate Content

First, we generate the content for our Instagram carousel post based on a request:

In [4]:
from datetime import datetime
today = datetime.now().strftime('%Y-%b-%d')
today

'2025-Apr-20'

In [5]:
# Generate content
request = f"Today is {today}. Generate a AI news Post pleaese. Agent especially. Just one news like new packages, new tech and etc"
content_json = agent.generate_content(request)

2025-04-20 18:05:59,753 - ig_agent - INFO - Generating content for request: Today is 2025-Apr-20. Generate a AI news Post pleaese. Agent especially. Just one news like new packages, new tech and etc
2025-04-20 18:06:08,388 - httpx - INFO - HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-20 18:06:09,473 - primp - INFO - response: https://html.duckduckgo.com/html 200
2025-04-20 18:06:25,466 - httpx - INFO - HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-20 18:06:40,017 - httpx - INFO - HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-20 18:06:40,026 - ig_agent.content_agent - INFO - Generated unique content on attempt 1


In [6]:
# Display the generated content structure
print(f"Cover: {content_json['cover']['heading_line1']} {content_json['cover']['heading_line2']}")
print(f"Hashtag: #{content_json['cover']['hashtag']}")
print(f"Number of content pages: {len(content_json['content_pages'])}")
print("\nContent Pages:")
for i, page in enumerate(content_json['content_pages']):
    print(f"  Page {i+1}: {page['title']} - {page['main_point']}")

print("\nCaption Preview:")
print(content_json['caption'][:200] + "...")

Cover: Fujitsu AI震撼登場 Kozuchi代理創新
Hashtag: #AIAgents
Number of content pages: 7

Content Pages:
  Page 1: 什麼是Kozuchi AI代理？ - Fujitsu的Kozuchi是開源框架，專為AI代理設計，能簡化工作流程並處理RAG任務。它於2024年10月發布，讓AI能自主決策，適用於企業自動化。
  Page 2: 主要功能與優勢 - Kozuchi支援低代碼開發，快速處理大量數據並預測趨勢。它的決策能力讓任務從幾週縮短到幾分鐘，提升效率並減少錯誤。
  Page 3: 實際應用場景 - 在企業中，Kozuchi可用於生成報告或監控系統，幫助預測市場變化。像是零售業能用它優化庫存，實現即時調整。
  Page 4: 與其他框架的比較 - 相較Langflow，Kozuchi更注重高階決策，提供更強的企業整合。2025年調查顯示，51%的公司已採用類似框架，提升生產力。
  Page 5: 未來影響與趨勢 - Kozuchi代表AI代理的未來，可能引領自動化革命。預計2025年底，更多產業將採用，改變工作方式。
  Page 6: 如何開始使用？ - 下載Fujitsu的Kozuchi框架，從簡單測試開始。結合數據源，就能建立自訂AI代理，提升你的工作效率。
  Page 7: 為何這很重要？ - 在AI快速進化的2025年，Kozuchi讓你輕鬆跟上潮流，提供實用工具，幫助職業發展和創新。

Caption Preview:
你知道嗎？在2025年4月，Fujitsu推出Kozuchi AI代理，這是AI領域的一大突破，能讓企業決策更快、更智能！這不是科幻，而是現實科技，能處理海量數據並自動解決問題。從這篇貼文，你將學到Kozuchi的核心功能、實際應用，以及它如何改變產業。準備好探索AI的未來了嗎？

首先，Kozuchi是Fujitsu在2024年10月全球發布的AI代理框架，專注於高階決策和問題解決。它整合了RA...


### 2.2 Generate Images

Next, we generate the images for our carousel post based on the content:

In [7]:
import os
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')

In [8]:
# Generate images
images = agent.generate_images(content_json)

# Display summary
print(f"Generated {len(images)} images:")
for img in images:
    print(f"  {img['type'].capitalize()}{' ' + str(img['page_number']) if 'page_number' in img else ''}: {img['file_name']}")

2025-04-20 18:06:40,054 - ig_agent - INFO - Generating images from content
2025-04-20 18:06:40,185 - ig_agent.template_processor - WARNING - PyCairo is not available, falling back to other methods
2025-04-20 18:06:40,185 - ig_agent.template_processor - INFO - CairoSVG version: 2.7.1
2025-04-20 18:06:40,186 - ig_agent.template_processor - INFO - Attempting SVG conversion with rsvg-convert for cover...
2025-04-20 18:06:40,479 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-20 18:06:40,480 - ig_agent.template_processor - INFO - Generated cover image: output/2025-04-20/20250420_180559/cover.png
2025-04-20 18:06:40,483 - ig_agent.template_processor - INFO - Attempting SVG conversion with rsvg-convert for content page 1...
2025-04-20 18:06:40,609 - ig_agent.template_processor - INFO - Successfully converted SVG to PNG using rsvg-convert
2025-04-20 18:06:40,610 - ig_agent.template_processor - INFO - Generated content image 1: output/2025-04-

Generated 8 images:
  Cover: cover.png
  Content 1: content_01.png
  Content 2: content_02.png
  Content 3: content_03.png
  Content 4: content_04.png
  Content 5: content_05.png
  Content 6: content_06.png
  Content 7: content_07.png


### 2.3 Display Generated Images

Now, let's visualize the images that were generated:

In [9]:
# # Display cover image
# cover_image = next(img for img in images if img['type'] == 'cover')
# print("Cover Image:")
# display(Image(cover_image['path']))

In [10]:
# # Display cover image
# cover_image = next(img for img in images if img['type'] != 'cover')
# print("Cover Image:")
# display(Image(cover_image['path']))

### 2.4 Send Notification (Optional)

If you've provided email credentials, you can send a notification with the generated content:

In [11]:
# Send notification if credentials are available
if os.environ.get("EMAIL_USER") and os.environ.get("EMAIL_PASSWORD"):
    recipient_email = "your.email@example.com"  # Replace with your email
    notification_status = agent.send_notification(recipient_email)
    print(f"Notification sent: {notification_status['sent']}")
    if notification_status['sent']:
        print(f"Email sent to: {notification_status['recipient']}")
        print(f"Subject: {notification_status['subject']}")
else:
    print("Email credentials not available, skipping notification")

Email credentials not available, skipping notification


### 2.5 Post to Instagram (Optional)

If you've provided Instagram credentials, you can post the content to Instagram:

In [12]:
# Post to Instagram if credentials are available
if os.environ.get("INSTAGRAM_USERNAME") and os.environ.get("INSTAGRAM_PASSWORD"):
    try:
        # Try with explicit error handling
        instagram_status = agent.post_to_instagram()
        print(f"Posted to Instagram: {instagram_status['posted']}")
        
        if instagram_status['posted']:
            print(f"Post URL: {instagram_status.get('url', 'N/A')}")
            print(f"Post type: {instagram_status.get('type', 'N/A')}")
        else:
            print(f"Posting failed: {instagram_status.get('error', 'Unknown error')}")
            print(f"Suggestion: {instagram_status.get('suggestion', 'Check logs for details')}")
            
            # If simulated, show what would have been posted
            if instagram_status.get('simulated', False):
                print("\nSimulated posting:")
                print(f"Would have posted {instagram_status.get('num_images', 0)} images")
                print(f"With caption length: {len(instagram_status.get('caption', ''))}")
    except Exception as e:
        print(f"Exception during Instagram posting: {str(e)}")
        print("Check that your Instagram credentials are correct and that the instagrapi package is installed")
        print("Install with: pip install instagrapi")
else:
    print("Instagram credentials not available, skipping post")
    print("To post to Instagram, set the INSTAGRAM_USERNAME and INSTAGRAM_PASSWORD environment variables")

2025-04-20 18:06:41,427 - ig_agent - INFO - Posting to Instagram
2025-04-20 18:06:41,428 - instagrapi - INFO - https://i.instagram.com/api/v1/launcher/sync/
2025-04-20 18:06:42,452 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/launcher/sync/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-20 18:06:43,342 - instagrapi - INFO - https://i.instagram.com/api/v1/accounts/login/
2025-04-20 18:06:46,639 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/accounts/login/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-20 18:06:47,646 - instagrapi - INFO - https://i.instagram.com/api/v1/feed/reels_tray/
2025-04-20 18:06:48,318 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com/api/v1/feed/reels_tray/ (269.0.0.18.75, OnePlus 6T Dev)
2025-04-20 18:06:49,326 - instagrapi - INFO - https://i.instagram.com/api/v1/feed/timeline/
2025-04-20 18:06:51,330 - private_request - INFO - datasci_daily [200] POST https://i.instagram.com

Posted to Instagram: True
Post URL: https://www.instagram.com/p/DIqhh9IR3GK/
Post type: single


## 3. Using the LangGraph Pipeline

The Instagram Agent can also be used with a LangGraph pipeline, which provides a graph-based workflow with streaming intermediate results.

In [13]:
from ig_agent.pipeline import create_pipeline

# Create the pipeline
pipeline = create_pipeline(
    llm=llm,
    output_dir=output_dir,
    email_user=os.environ.get("EMAIL_USER"),
    email_password=os.environ.get("EMAIL_PASSWORD")
)

In [ ]:
# # Visualize the pipeline graph
# from IPython.display import display, Image

# # Create the pipeline
# pipeline = create_pipeline(
#     llm=llm,
#     output_dir=output_dir,
#     email_user=os.environ.get("EMAIL_USER"),
#     email_password=os.environ.get("EMAIL_PASSWORD")
# )

# # Visualize the graph
# try:
#     display(Image(pipeline.get_graph().draw_mermaid_png()))
# except Exception as e:
#     print(f"Error displaying graph: {e}")
#     print("If you can't see the graph, make sure graphviz is installed.")

### 3.1 Visualize the Pipeline Graph

Let's visualize the pipeline graph to understand the workflow:

In [ ]:
# Try to visualize the pipeline graph
try:
    # Visualize the graph (requires graphviz)
    display(Image(pipeline.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Unable to visualize graph: {e}")
    print("Consider installing graphviz to visualize the graph")
    # Create a simple representation of the graph
    html = """
    <div style="border: 1px solid #ccc; padding: 10px;">
    <h3>Pipeline Graph</h3>
    <pre>
    START → supervisor → [ content_agent, image_agent, notification_agent ] → END
    
    Flow:
    1. The supervisor routes tasks to the appropriate agent
    2. Content agent generates content
    3. Image agent creates visuals
    4. Notification agent sends emails
    5. The supervisor determines when to finish
    </pre>
    </div>
    """
    display(HTML(html))

### 3.2 Run the Pipeline with Streaming

Run the pipeline with streaming to see the intermediate results:

In [ ]:
# Set up inputs for the pipeline
inputs = {
    "messages": [
        {
            "role": "user", 
            "content": "Generate a carousel post about Python data analysis libraries"
        }
    ]
}

# Process and display streaming results
for output in pipeline.stream(inputs):
    if "__end__" not in output:
        step = list(output.keys())[-1]
        print(f"\n--- Step: {step} ---")
        
        # Display agent messages
        if step == "supervisor":
            supervisor_output = output[step]
            if "next" in supervisor_output:
                print(f"Supervisor decision: Next agent = {supervisor_output['next']}")
        elif step == "content_agent":
            print(output[step]["messages"][-1].content)
            if "content_json" in output[step]:
                content_json = output[step]["content_json"]
                print(f"Generated content with {len(content_json.get('content_pages', []))} pages")
        elif step == "image_agent":
            print(output[step]["messages"][-1].content)
            if "images" in output[step]:
                images = output[step]["images"]
                print(f"Generated {len(images)} images")
        elif step == "notification_agent":
            print(output[step]["messages"][-1].content)
            if "notification_status" in output[step]:
                print(f"Notification status: {output[step]['notification_status']}")

print("\nPipeline execution completed.")

### 3.3 Display the Final Results

After the pipeline has completed, we can display the generated images:

In [ ]:
# Import glob to find the generated images
import glob

# Find the PNG files in the output directory
png_files = sorted(glob.glob(str(output_dir / "*.png")))

if png_files:
    print(f"Found {len(png_files)} images:")
    for img_path in png_files:
        print(f"\nImage: {os.path.basename(img_path)}")
        display(Image(img_path))
else:
    print("No images found in the output directory.")

## 4. Full Pipeline in One Go

You can also run the entire pipeline in one go using the `run_pipeline()` method:

In [ ]:
# # Create a new output directory for this run
# import datetime
# # We only need to specify a base directory now
# # DateTime subdirectories will be created automatically
# new_output_dir = Path(f"notebook_output_run")
# new_output_dir.mkdir(exist_ok=True)

# # Create a new agent instance
# new_agent = InstagramAgent(
#     llm=llm,
#     output_dir=new_output_dir,
#     email_user=os.environ.get("EMAIL_USER"),
#     email_password=os.environ.get("EMAIL_PASSWORD")
# )

# # Run the complete pipeline
# request = "Generate a carousel post about AI-powered data visualization techniques"
# recipient_email = "your.email@example.com"  # Replace with your email if needed

# # Run with or without notification based on credentials
# if os.environ.get("EMAIL_USER") and os.environ.get("EMAIL_PASSWORD"):
#     results = new_agent.run_pipeline(request, recipient_email=recipient_email)
# else:
#     results = new_agent.run_pipeline(request)

# # Display results summary
# print("Pipeline execution completed.")
# print(f"Output directory: {results['output_dir']}")
# print(f"Content pages: {len(results['content']['content_pages'])}")
# print(f"Images generated: {len(results['images'])}")
# if 'notification' in results:
#     print(f"Notification sent: {results['notification']['sent']}")

### 4.1 Display the Results from the Full Pipeline Run

In [ ]:
# # Display the images from the full pipeline run
# for img in results['images']:
#     print(f"\n{img['type'].capitalize()}{' ' + str(img['page_number']) if 'page_number' in img else ''}:")
#     display(Image(img['path']))

## 5. New Features

The `ig_agent` library includes several key features that improve its functionality:

1. **Content Line Breaks**: Content text is now broken into lines with approximately 18 words per line, making the text more readable on Instagram carousel posts.

2. **DateTime-based Output Directories**: Output files are now organized in a hierarchical directory structure:
   - Base directory (e.g., "notebook_output")
   - Date subfolder (e.g., "2025-04-20")
   - Timestamp subfolder (e.g., "20250420_154900")

This organization makes it easier to track and manage multiple content generation runs and preserves a history of generated content.

## 6. Summary

The `ig_agent` library provides a powerful multi-agent system for generating Instagram carousel posts. You can use it in several ways:

1. **Direct API**: Use the individual methods for fine-grained control over each step
2. **LangGraph Pipeline**: Use the graph-based workflow with streaming results
3. **One-step Pipeline**: Run the entire process in one go with `run_pipeline()`

The system generates structured content, converts it to visually appealing images using SVG templates, and optionally sends notifications or posts to Instagram.